# 先使用bs4爬取

In [5]:
from bs4 import BeautifulSoup

# 這裡假設 html_content 是你從該網頁獲得的HTML內容
html_content = """
<li _ngcontent-yby-c4=""><p _ngcontent-yby-c4="" class="date">2023-09-15 10:00</p><span _ngcontent-yby-c4="" style="cursor: pointer">遭通報之境外帳戶(更新至1120914)</span></li>

<li _ngcontent-yby-c4=""><p _ngcontent-yby-c4="" class="date">2023-08-15 14:45</p><span _ngcontent-yby-c4="" style="cursor: pointer">全家便利商店 推出 Fun心取爭議包裹退貨申請平臺</span></li>

<li _ngcontent-yby-c4=""><p _ngcontent-yby-c4="" class="date">2023-08-10 12:00</p><span _ngcontent-yby-c4="" style="cursor: pointer">出國前訂購網卡 接到訂單錯誤來電??</span></li>

<li _ngcontent-yby-c4=""><p _ngcontent-yby-c4="" class="date">2023-08-09 17:43</p><span _ngcontent-yby-c4="" style="cursor: pointer">小心釣魚簡訊，收到簡訊附網址請先查詢！</span></li>

<li _ngcontent-yby-c4=""><p _ngcontent-yby-c4="" class="date">2023-08-07 09:25</p><span _ngcontent-yby-c4="" style="cursor: pointer">【網站維護公告】112/08/08 11:30-13:30</span></li>
"""

soup = BeautifulSoup(html_content, 'lxml')  # 使用 lxml 解析器

# 提取所有的新聞日期和標題
news_data = []
for li in soup.find_all('li', {'_ngcontent-yby-c4': ''}):
    date = li.find('p', class_='date').text
    title = li.find('span').text
    news_data.append((date, title))

# 顯示結果
for date, title in news_data:
    print(date, title)

2023-09-15 10:00 遭通報之境外帳戶(更新至1120914)
2023-08-15 14:45 全家便利商店 推出 Fun心取爭議包裹退貨申請平臺
2023-08-10 12:00 出國前訂購網卡 接到訂單錯誤來電??
2023-08-09 17:43 小心釣魚簡訊，收到簡訊附網址請先查詢！
2023-08-07 09:25 【網站維護公告】112/08/08 11:30-13:30


# 使用selenium爬第一頁

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

URL = "https://165.npa.gov.tw/#/articles/1"

# 啟動瀏覽器並打開網頁
service = Service('chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service,options=options)
driver.get(URL)

# 等待網頁加載完成
wait = WebDriverWait(driver, 20)  # 增加到 20 秒

# 使用明確的方法查找元素
try:
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li > p.date')))

    # 獲取所有相匹配的元素
    news_elements = driver.find_elements(By.CSS_SELECTOR, '.articleList > li')

    if not news_elements:
        print("未找到任何匹配的新聞元素。")
    else:
        for news in news_elements:
            date = news.find_element(By.CSS_SELECTOR, 'p.date').text
            title = news.find_element(By.CSS_SELECTOR, 'span').text
            print(f"{date} {title}")

except Exception as e:
    print(f"發生錯誤: {e}")

finally:
    # 關閉瀏覽器
    driver.quit()

2023-09-15 10:00 遭通報之境外帳戶(更新至1120914)
2023-08-15 14:45 全家便利商店 推出 Fun心取爭議包裹退貨申請平臺
2023-08-10 12:00 出國前訂購網卡 接到訂單錯誤來電??
2023-08-09 17:43 小心釣魚簡訊，收到簡訊附網址請先查詢！
2023-08-07 09:25 【網站維護公告】112/08/08 11:30-13:30


# 使用selenium爬第一頁+內文

In [23]:
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

URL = "https://165.npa.gov.tw/#/articles/1"
current_date = datetime.now().strftime("%Y%m%d")

# 啟動瀏覽器並打開網頁
service = Service('chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service,options=options)
driver.get(URL)

# 等待網頁加載完成
wait = WebDriverWait(driver, 10)  
df = pd.DataFrame(columns=["日期", "新聞標題", "內文"])

# 使用明確的方法查找元素
try:
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li > p.date')))
    
    news_count = len(driver.find_elements(By.CSS_SELECTOR, '.articleList > li'))
    
    for index in range(news_count):
        # 每次迭代都重新找到新聞元素，避免 stale element problems
        news_elements = driver.find_elements(By.CSS_SELECTOR, '.articleList > li')
        news = news_elements[index]
        
        date = news.find_element(By.CSS_SELECTOR, 'p.date').text
        title_element = news.find_element(By.CSS_SELECTOR, 'span')
        title = title_element.text

        # 模擬點擊事件
        title_element.click()

        try:
            # 等待頁面中的文字內容加載完成，最多等待10秒
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div p span')))

            # 提取新聞內文
            content_elements = driver.find_elements(By.CSS_SELECTOR, 'div p > span')
            content = ' '.join([elem.text for elem in content_elements if not elem.find_elements(By.CSS_SELECTOR, "img")])
            
        except TimeoutException:
            # 如果超過10秒還沒找到元素，設定內容為 "NA"
            content = "NA"

        print(f"{date} {title}:\n{content}\n")
        
        # 將新聞資料添加到DataFrame中
        df.loc[len(df)] = [date, title, content]

        # 返回到主頁面
        driver.back()
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li > p.date')))  # 確保新聞元素再次出現

except Exception as e:
    print(f"發生錯誤: {e}")

# 保存DataFrame到Excel文件
df.to_excel(f'{current_date}_165新聞快訊爬蟲.xlsx', index=False, engine='openpyxl')

# 關閉瀏覽器
driver.quit()

2023-09-15 10:00 遭通報之境外帳戶(更新至1120914):
NA

2023-08-15 14:45 全家便利商店 推出 Fun心取爭議包裹退貨申請平臺:
什麼是一頁式廣告跨境包裹詐騙? 詐騙集團利用社群平臺創立假粉絲專頁投放廣告，以低於市場行情之價格促銷各類商品，吸引民眾目光，且此類廣告以「貨到付款」方式取信民眾，等到消費者收貨付款後卻發現商品與介紹不符，品質低劣。 那...付出去的錢怎麼拿回來?! 全家便利商店於今年7月初推出爭議包裹退貨申請平臺服務，民眾線上申請後，帶著原包裹到任一家全家超商印單退貨。廠商收到退貨 後，進行退貨包裹內容確認，確認後將於10日內完成退款，期降低民眾自行聯繫之困擾。 申請平臺及詳細說明 : https://returns.com.tw/web/index.php 全家平臺可接受處理的『詐騙包裹』定義如下： 1、消費者並未下單，卻收到到貨簡訊並至全家店舖完成繳費取貨。 2、消費者取貨後發現為空包裹，並提供相關資料佐證。 3、其他問題將視為『消費糾紛』，將依申訴內容受理，但不保障退款，該平台會協助通知廠商處理。 最重要的，還是認明一頁式廣告特徵，才可以免除後續困擾!

2023-08-10 12:00 出國前訂購網卡 接到訂單錯誤來電??:
疫情解封後，國人開始有出國旅遊的規劃 近期有民眾通報表示接獲詐騙集團假冒「JOYTEL卓一電訊」名義，稱之前訂購網卡、SIM卡，因訂單操作錯誤，需操作ATM或網路銀行以解除分期付款。 請注意：如接獲來電顯示開頭帶「+」號、「+1」、「+886」等號碼，電話中如聽到關鍵字「解除重複扣款」、「誤設會員等級」、「登記批發商」等關鍵字，請立即掛斷，並將相關資訊先向電商業者反映，如有被害請通報165反詐騙諮詢專線。

2023-08-09 17:43 小心釣魚簡訊，收到簡訊附網址請先查詢！:
若您收到簡訊，內容是水費、電費、汽燃費逾期，要求您點擊短網址連結進行繳費??? 請小心!!! 不要點擊連結，小心個資遭盜 或 信用卡被盜刷。 收到要求繳費之簡訊，請先上各單位官網查詢真偽，避免遭詐。 臺灣自來水公司 https://www.water.gov.tw/ 臺灣電力股份有限公司 https://www.taipower.com.tw/tc/index.aspx 監理服務網 https://ww

# 抓好幾頁

In [2]:
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

URL = "https://165.npa.gov.tw/#/articles/1"
current_date = datetime.now().strftime("%Y%m%d")

service = Service('chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(URL)

wait = WebDriverWait(driver, 10)
df = pd.DataFrame(columns=["日期", "新聞標題", "內文"])

def scrape_page():
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li > p.date')))
        news_count = len(driver.find_elements(By.CSS_SELECTOR, '.articleList > li'))
        for index in range(news_count):
            news_elements = driver.find_elements(By.CSS_SELECTOR, '.articleList > li')
            news = news_elements[index]
            date = news.find_element(By.CSS_SELECTOR, 'p.date').text
            title_element = news.find_element(By.CSS_SELECTOR, 'span')
            title = title_element.text
            title_element.click()
            try:
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div p span')))
                content_elements = driver.find_elements(By.CSS_SELECTOR, 'div p > span')
                content = ' '.join([elem.text for elem in content_elements if not elem.find_elements(By.CSS_SELECTOR, "img")])
            except TimeoutException:
                content = "N/A"
            print(f"{title}")
            df.loc[len(df)] = [date, title, content]
            driver.back()
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li > p.date')))
    except Exception as e:
        print(f"發生錯誤: {e}")

# 爬取首頁的新聞
scrape_page()

#設定要爬幾篇
page = 15

# 爬取第幾頁的新聞
for i in range(2, page+1):  
    try:
        page_button = driver.find_element(By.XPATH, f"//li/a[text()='{i}']")
        page_button.click()
        scrape_page()
    except Exception as e:
        print(f"Error navigating to page {i}: {e}")
    
df.to_excel(f'{current_date}_{page}頁_165新聞快訊.xlsx', index=False, engine='openpyxl')

遭通報之境外帳戶(更新至1120914)
全家便利商店 推出 Fun心取爭議包裹退貨申請平臺
出國前訂購網卡 接到訂單錯誤來電??
小心釣魚簡訊，收到簡訊附網址請先查詢！
【網站維護公告】112/08/08 11:30-13:30
【全民齊反詐】1000個正忠排骨飯等你來抽獎！
常見詐欺六大手法
小心! 【社群影音平臺】也暗藏遭詐騙風險!
我們要的就是你 !!!
我國網路詐欺被害調查與防制研究
見面再談，匯錢? 免談!
防制詐騙簡訊民眾有感 半年減少近9成
遇到社群平臺一頁式廣告詐騙? 申請退款流程說明!
演唱會沒看到，還被騙錢!!
你被詐騙話術了嗎?
每人普發現金6000元方案不會用簡訊通知 請勿點擊不明連結
假冒銀行信用貸款簡訊
虛擬遊戲詐騙手法大公開
個資防護，從密碼開始
查證新管道!165全民防騙網可以線上查詢詐騙資訊囉
一點就詐!小心釣魚簡訊讓手機中毒!
網路求職、借貸 2 No! No!
111.11.11購物節，我們搶先推出!!!!
網路求職 一堆騙子! 小心社群平臺求職詐騙
沒認清投資詐騙誘惑，小心誤上賊船！！！！！
一分錢一分貨，小心奧梨子假蘋果
假紓困、真詐財，假冒官網網站激增!
幫好友輔助認證? LINE帳號會被盜!
安卓用戶照過來 165教您 開啟垃圾訊息阻擋功能阻擋詐騙簡訊
iPhone用戶照過來 165教您 阻絕透過iMessage 發送的詐騙簡訊
112/9/6-112/9/12民眾通報假投資(博弈)詐騙網站 【網友不會幫你賺錢、請勿聽信網友投資】
112/9/4~112/9/10詐騙來電排名
112/8/29-112/9/5民眾通報假投資(博弈)詐騙網站 【網友不會幫你賺錢、請勿聽信網友投資】
112/8/28-112/9/3詐騙來電排名
112/8/22-112/8/28民眾通報假投資(博弈)詐騙網站 【網友不會幫你賺錢、請勿聽信網友投資】
112/8/21-112/8/27詐騙來電排名
112/8/15-112/8/21民眾通報假投資(博弈)詐騙網站 【網友不會幫你賺錢、請勿聽信網友投資】
112/8/14-112/8/20詐騙來電排名
112/8/8-112/8/14民眾通報假投資(博弈)詐騙網站 【網友不會幫你賺錢、請勿聽信網友投資】
112/8/7-112/8/13詐騙來電排名
112/8/1-112/8/7民眾通報假投資(